# Word2vec model development

In [ ]:
import numpy as np
import pandas as pd
import gensim
import re

In [ ]:
## Downloading datasets
!wget -q https://www.dropbox.com/s/0ygoimffauvl7x5/unlabeledTrainData.tsv
!wget -q https://www.dropbox.com/s/4f1s02mh6bfjcr5/labeledTrainData.tsv

In [ ]:
df = pd.read_csv("unlabeledTrainData.tsv",sep='\t',quoting=3)
df.shape

(50000, 2)

In [ ]:
df.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


In [ ]:
df['review'][0]

'"Watching Time Chasers, it obvious that it was made by a bunch of friends. Maybe they were sitting around one day in film school and said, \\"Hey, let\'s pool our money together and make a really bad movie!\\" Or something like that. What ever they said, they still ended up making a really bad movie--dull story, bad script, lame acting, poor cinematography, bottom of the barrel stock music, etc. All corners were cut, except the one that would have prevented this film\'s release. Life\'s like that."'

In [ ]:
def clean_data(doc):
  doc = re.sub("<br"," ",doc) # replacing <br with whitespace
  doc = re.sub("[^A-Za-z]"," ",doc) # replacing every non alphabet with whitespace
  doc = " ".join([w.strip() for w in doc.strip().lower().split()]) # removing whitespace and converting to lowercase
  return doc

In [ ]:
df.review= df.review.apply(clean_data)
df.review[0]

In [ ]:
df.head()

## word2vec model

In [ ]:
corp = df.review.tolist()
print(len(corp))

50000


In [ ]:
data = [i.split(" ") for i in corp]
print(len(data))

50000


In [ ]:
print(data[0])

In [ ]:
model = gensim.models.Word2Vec(sentences=data,size=100,workers=8,min_count=12)

In [ ]:
model.wv.vectors.shape

(25706, 100)

In [ ]:
model.wv['king']

AttributeError: ignored

In [ ]:
model.wv['king'].shape

(100,)

In [ ]:
model.wv['seohghoeshgre']

In [ ]:
model.wv.most_similar("awesome")

[('amazing', 0.8112810850143433),
 ('incredible', 0.7490220069885254),
 ('excellent', 0.7278615236282349),
 ('fantastic', 0.6910278797149658),
 ('exceptional', 0.6683927774429321),
 ('great', 0.6500455737113953),
 ('cool', 0.6491718292236328),
 ('outstanding', 0.6489181518554688),
 ('alright', 0.6269726753234863),
 ('awful', 0.62667316198349)]

In [ ]:
model.save("imdb_dataset.vec")

In [ ]:
gensim.__version__

'3.6.0'

## Text classification

In [ ]:
df = pd.read_csv("/content/labeledTrainData.tsv",sep='\t',quoting=3)
df.shape

(25000, 3)

In [ ]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
df.review = df.review.apply(clean_data)
df.head()

,id,sentiment,review
0,"""5814_8""",1,with all this stuff going down at the moment w...
1,"""2381_9""",1,the classic war of the worlds by timothy hines...
2,"""7759_3""",0,the film starts with a manager nicholas bell g...
3,"""3630_4""",0,it must be assumed that those who praised this...
4,"""9495_8""",1,superbly trashy and wondrously unpretentious s...


In [ ]:
#split data into trian and test set
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts = train_test_split(df.review,df.sentiment,test_size=0.2,random_state=5)
print(xtr.shape,xts.shape,ytr.shape,yts.shape)


(20000,) (5000,) (20000,) (5000,)


### text data prepraration

In [ ]:
from tensorflow.keras.preprocessing import text
# create a tokenization object
tok = text.Tokenizer(num_words=20000)
tok.fit_on_texts(xtr.tolist())

In [ ]:
print(tok.word_index)

{'the': 1, 'and': 2, 'a': 3, 'of': 4, 'to': 5, 'is': 6, 'it': 7, 'in': 8, 'i': 9, 'this': 10, 'that': 11, 's': 12, 'was': 13, 'as': 14, 'for': 15, 'with': 16, 'movie': 17, 'but': 18, 'film': 19, 'you': 20, 't': 21, 'on': 22, 'not': 23, 'he': 24, 'are': 25, 'his': 26, 'have': 27, 'be': 28, 'one': 29, 'all': 30, 'at': 31, 'they': 32, 'by': 33, 'an': 34, 'who': 35, 'so': 36, 'from': 37, 'like': 38, 'there': 39, 'her': 40, 'or': 41, 'just': 42, 'about': 43, 'out': 44, 'if': 45, 'has': 46, 'what': 47, 'some': 48, 'good': 49, 'can': 50, 'she': 51, 'more': 52, 'when': 53, 'very': 54, 'up': 55, 'even': 56, 'no': 57, 'time': 58, 'would': 59, 'my': 60, 'which': 61, 'story': 62, 'only': 63, 'really': 64, 'see': 65, 'their': 66, 'had': 67, 'me': 68, 'we': 69, 'were': 70, 'well': 71, 'than': 72, 'much': 73, 'people': 74, 'get': 75, 'been': 76, 'will': 77, 'bad': 78, 'do': 79, 'other': 80, 'also': 81, 'into': 82, 'great': 83, 'because': 84, 'first': 85, 'him': 86, 'how': 87, 'don': 88, 'most': 89, '

In [ ]:
for i in range(5):
  print(len(xtr[i].split(' ')))

437
161
375
384
379


In [ ]:
# text to sequence conversion
xtr = tok.texts_to_sequences(xtr.tolist())
xts = tok.texts_to_sequences(xts.tolist())

In [ ]:
print(xtr[0])

[7, 12, 247, 5, 347, 10, 17, 8, 447, 749, 281, 480, 64, 71, 126, 1, 947, 113, 224, 2, 615, 25, 30, 85, 927, 15, 1, 15860, 7, 12, 3, 720, 1070, 62, 4, 116, 2, 396, 3041, 15, 1, 19, 3665, 39, 12, 906, 4, 2432, 3066, 2, 1781, 4082, 39, 25, 48, 17700, 15076, 2, 14, 46, 76, 7069, 22, 1, 7886, 1, 275, 1456, 1813, 45, 29, 6, 10216, 16, 1, 3387, 1490, 2311, 2568, 187, 9, 547, 1, 275, 14, 110, 52, 720, 670, 5, 1, 190, 11, 243, 72, 6706, 1, 3387, 1490, 12, 1441, 8048, 663, 26, 1255, 5, 1, 2027, 202, 1042, 1, 3519, 38, 1803, 306, 8048, 462, 14409, 143, 26, 3491, 729, 7, 164, 165, 282, 41, 23, 8, 100, 413, 9, 2628, 1, 190, 11, 8048, 16, 3871, 12777, 3492, 2, 42, 15861, 1, 78, 477, 29, 33, 29, 8, 26, 18802, 2691, 15, 1326, 61, 164, 1896, 8, 3041, 199, 15, 306, 2, 1, 1319, 8, 100, 413, 10, 19, 6, 54, 73, 287, 147, 45, 20, 151, 31, 30, 3493, 5, 1, 497, 34, 313, 775, 83, 472, 3666, 947, 2, 1129, 359, 605, 1, 2049, 93, 10, 3, 347, 41, 127, 19]


206
90
118
425
207


In [ ]:
# resize each document to a same size
from tensorflow.keras.preprocessing import sequence
xtr = sequence.pad_sequences(xtr,maxlen=250,padding='post')
xts = sequence.pad_sequences(xts,maxlen=250,padding='post')

In [ ]:
for i in xtr[:5]:
  print(len(i))

250
250
250
250
250


In [ ]:
wvmodel = gensim.models.Word2Vec.load("/content/imdb_dataset.vec")

In [ ]:
wvmodel.wv.vectors.shape

(25706, 100)

In [ ]:
custom_wv_embedding = np.zeros((20001,100))

for word,i in sorted(tok.word_index.items(),key=lambda x:x[1]):
  if i>20000:
    break
  if word in wvmodel.wv.vocab:
    custom_wv_embedding[i] = wvmodel.wv[word]

In [ ]:
custom_wv_embedding.shape

(20001, 100)

In [ ]:
wvmodel.wv['king']

In [ ]:
xtr.shape

(20000, 250)

## Modelling

In [ ]:
from tensorflow.keras import models,layers

In [ ]:
input_layer = layers.Input(shape=(250,))
# adding an embedding layer specifying total unique input words as input dim, size of vector as output dim
em_layer = layers.Embedding(input_dim=20001,output_dim=100,weights=[custom_wv_embedding],trainable=False)(input_layer)
ft = layers.Flatten()(em_layer)
h1 = layers.Dense(400,activation='relu')(ft)
h2 = layers.Dense(200,activation='relu')(h1)
h3 = layers.Dense(80,activation='relu')(h2)
op = layers.Dense(1,activation='sigmoid')(h3)
model = models.Model(inputs=input_layer,outputs=op)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 250, 100)          2000100   
_________________________________________________________________
flatten_2 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 400)               10000400  
_________________________________________________________________
dense_5 (Dense)              (None, 200)               80200     
_________________________________________________________________
dense_6 (Dense)              (None, 80)                16080     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 81  

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(xtr,ytr,epochs=10,batch_size=100,validation_data=(xts,yts))

Epoch 1/10
200/200 [==============================] - 25s 118ms/step - loss: 0.5960 - accuracy: 0.7025 - val_loss: 0.4583 - val_accuracy: 0.7814
Epoch 2/10
200/200 [==============================] - 23s 117ms/step - loss: 0.2911 - accuracy: 0.8766 - val_loss: 0.5156 - val_accuracy: 0.7646
Epoch 3/10
200/200 [==============================] - 23s 117ms/step - loss: 0.1075 - accuracy: 0.9596 - val_loss: 0.7118 - val_accuracy: 0.7716
Epoch 4/10
200/200 [==============================] - 23s 117ms/step - loss: 0.0609 - accuracy: 0.9765 - val_loss: 0.9325 - val_accuracy: 0.7788
Epoch 5/10
200/200 [==============================] - 23s 117ms/step - loss: 0.0320 - accuracy: 0.9885 - val_loss: 1.1038 - val_accuracy: 0.7698
Epoch 6/10
200/200 [==============================] - 23s 117ms/step - loss: 0.0262 - accuracy: 0.9912 - val_loss: 1.1847 - val_accuracy: 0.7720
Epoch 7/10
200/200 [==============================] - 24s 118ms/step - loss: 0.0275 - accuracy: 0.9910 - val_loss: 1.2336 - val_ac

In [ ]:
x = [8,7,5,6,2,6,3,2,5,4,3]
sorted(x)

[2, 2, 3, 3, 4, 5, 5, 6, 6, 7, 8]

In [ ]:
y = ['hello','hi','anshu','boy','cat','zebra']
sorted(y)

['anshu', 'boy', 'cat', 'hello', 'hi', 'zebra']

In [ ]:
y = [('hello',6),('hi',9),('anshu',3),('boy',2),('cat',7),('zebra',4)]
sorted(y)

[('anshu', 3), ('boy', 2), ('cat', 7), ('hello', 6), ('hi', 9), ('zebra', 4)]

In [ ]:
sorted(y,key=lambda x:x[1])

[('boy', 2), ('anshu', 3), ('zebra', 4), ('hello', 6), ('cat', 7), ('hi', 9)]